In [7]:
import os
import pandas as pd

dfs = []
for filename in os.listdir('data'):
    path = os.path.join('data', filename)
    if os.path.isfile(path):
        df = pd.read_csv(path)
        if len(df) >= 999:
            dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [30]:
df.groupby('postcode').min('date')

,id,date,isRural,landSize,isRetirement
postcode,,,,,
810,2012972258,20201231,False,0.0,False
812,2011513680,20161222,False,0.0,False
820,2011191856,20200128,False,0.0,False
830,2008913986,20180204,False,0.0,False
832,2013354993,20190214,False,0.0,False
...,...,...,...,...,...
3193,2014496269,20200229,False,0.0,False
3351,2011320025,20171108,False,0.0,False
3555,2012349472,20210111,False,0.0,False
